# Intialisation of the process on Google Colab

In [1]:
# Installation of Google Drive
from google.colab import drive
drive.mount("/content/gdrive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
# Cloning of the GitHub repository onto Google Drive
! git clone https://github.com/GSacchetti/Segmentation-of-multispectral-and-thermal-images-with-U-NET.git

In [3]:
# Access to the folder on google Drive, if it doesn't work, access the pannel on the left, search for the repository and copy the path
%cd gdrive/My Drive/U-NET

/content/gdrive/My Drive/U-NET


In [ ]:
# See all the folders contained in the repository
!ls

Construction_data		   model_unet_bis.py
data				   model_unet.py
data_normalizing.py		   pred_unet.py
data_Preparation_Instructions.txt  __pycache__
EVALUATIONS			   rafik-msk-rcnn.tgz
load_train_test_data.py		   README.md
main.py				   requirements.txt
Mask_handling			   Train_Test_Instructions.txt
measurment_functions.py		   VERIFICATIONS
Models


In [4]:
# Installation of the requirements
!pip install -r requirements.txt

In [5]:
# Install a new version of xlrd
!pip install xlrd==1.2

     |████████████████████████████████| 112kB 2.8MB/s 
  Found existing installation: xlrd 1.1.0
    Uninstalling xlrd-1.1.0:
      Successfully uninstalled xlrd-1.1.0


In [6]:
# Verify that the right version has been installed (1.2.0)
import xlrd
print(xlrd.__VERSION__)


1.2.0


# Initialisation of the GPU



To make sure you're borrowing a GPU from Google :
  Go to "Edit" -> "Notebook settings" -> "Hardware accelerators" change it from "None" to "GPU" and finally "Save". 
  
  Once this is done, run the three following cells to make sure the GPU is working correctly

In [7]:
# This cell return the name of the GPU (/device:GPU:0)
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [8]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 3824368803994855912, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 3076511982207590938
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 9328176753869995924
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 11146783616
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 4747101635079811641
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"]

In [9]:
!cat /proc/meminfo

MemTotal:       13333552 kB
MemFree:         9471372 kB
MemAvailable:   12043796 kB
Buffers:           80788 kB
Cached:          2556860 kB
SwapCached:            0 kB
Active:          1116020 kB
Inactive:        2380672 kB
Active(anon):     747020 kB
Inactive(anon):     2396 kB
Active(file):     369000 kB
Inactive(file):  2378276 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:              2088 kB
Writeback:             0 kB
AnonPages:        858996 kB
Mapped:           539348 kB
Shmem:              3016 kB
Slab:             177496 kB
SReclaimable:     131940 kB
SUnreclaim:        45556 kB
KernelStack:        4776 kB
PageTables:         8476 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:     6666776 kB
Committed_AS:    3495524 kB
VmallocTotal:   34359738367 kB
VmallocUsed:           0 kB
VmallocChunk:          0 kB
Percpu:              952 kB
AnonHugePages:   

# Data Preparation

In [ ]:
# See the code for the transformation from excel to tif
!cat Construction_data/excel_to_tif_array.py

In [ ]:
# Run the code to create the tif images
%run Construction_data/excel_to_tif_array.py

In [ ]:
# See the code of create_train_test_data
!cat Construction_data/create_train_test_data.py

In [ ]:
# Split of the dataset 80% for training and 20% for verification
%run Construction_data/create_train_test_data.py

# Training of U-NET

In [ ]:
# See the function for loading the training and test datasets
!cat load_train_test_data.py

In [ ]:
# See the model for the U-NET method, in this file the learning rate, optimizer, loss function, number of epochs before early stopping
!cat model_unet.py

In [ ]:
# See the training algorithm for U-NET, in this file, it is possible to change the channels taken into account for the learning, as well as if it should load the weight or create
# a new weight file
!cat main.py

#This program is for loading data and training the U-NET model

from PIL import Image
import matplotlib.pyplot as plt
import numpy  as np
import math
import os
import sys
import random
from skimage.io import imsave,imread

import time
import datetime

import model_unet
import measurment_functions
import data_normalizing
import load_train_test_data


from tensorflow import keras
from keras.models import Model,load_model,save_model
from keras.layers import Input
from keras.layers.core import Dropout
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from os.path import join


def mymkdir(path):
    if not os.path.exists(path):
        os.mkdir(path)

# Learning function of the U-NET model

def Training_Unet(mainPath='',Augmentation=True,nb_Augmentation=3,INPUT_WIDTH = 256,IN

In [ ]:
# Run the training algorithm, it will first load the datasets, then summarize the model and finally start training.
# To change the channels used simply add : "[4,5]" after %run main.py.
# Example :
#%run main.py "[0,1,2]"
%run main.py

In [33]:
# See the pred_unet file to make prediction depending on the weight file
!cat pred_unet.py

#This script is for using the trained U-NET model, To make predictions on other images of our dataset.
# uploading packages 

from PIL import Image
import matplotlib.pyplot as plt
import numpy  as np
import math
import os
import sys
import ast
import random
from skimage.io import imsave,imread
import matplotlib.pyplot as plt

import time
import datetime

import model_unet 
import data_normalizing
import load_train_test_data
import measurment_functions

from os.path import join
from tensorflow import keras
from keras.models import Model,load_model,save_model
from keras.layers import Input
from keras.layers.core import Dropout
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator


# Load_weights = True : if we load the existing model that is already driven.
# validation_split,

IMPORTANT : Make sure the channels indicated in that file are the same ones as the one used for training.

Channels can be changed directly in the file as well as the weight file to use.

Multiple weight files are provided, trained on different combination of channels.

To do so, go to line 63 :
model = load_model('model-U_NET.h5')
and change 'model-U_NET.h5' by 'model-U_NET_[...].h5', with in the brackets the channels wanted. Check "!ls" at the beginning to review all the available weight files..

In [35]:
# To change the channels used simply add : "[4,5]" after %run pred_unet.py, also it is still needed to change the name of the weight file in the script according
# to the restult you want.
# Example :
#%run pred_unet.py "[0,1,2]"

# To make a prediction on one subset, use %run pred_unet.py "[0,1,2,3,4,5,6]" "npy" or %run pred_unet.py "[0,1,2,3,4,5,6]"
# To make predictions on the full Dalles contained in "/content/gdrive/My Drive/U-NET/data/Image_to_pred/DallesTif", use %run pred_unet "[0,1,2,3,4,5,6]" "tif"
%run pred_unet.py

/content/gdrive/My Drive/U-NET/pred_unet.py:84: UserWarning: /content/gdrive/My Drive/U-NET/VERIFICATIONS/concentric.png is a low contrast image
  imsave(filename_verif,concentric_weights)


/content/gdrive/My Drive/U-NET/data/Image_to_pred/prediction.log
576 1632
32 32
(1, 576, 1632, 7) (576, 1632, 7)
(1, 576, 1632, 7) (576, 1632, 7)
(1, 576, 1632, 7) (576, 1632, 7)
(1, 576, 1632, 7) (576, 1632, 7)
(1, 576, 1632, 7) (576, 1632, 7)
(1, 576, 1632, 7) (576, 1632, 7)
(1, 576, 1632, 7) (576, 1632, 7)
1/1 [==============================] - 0s 2ms/step


/content/gdrive/My Drive/U-NET/EVALUATIONS/0.5Image.npy.png


# Precision measurment 

In [ ]:
# Loading of the images and transformation in array

from PIL import Image
from os.path import join
import numpy as np

# Ground truth :
tif_path = join('/content/gdrive/My Drive/U-NET/data/Image_to_pred', 'Image_pred.tif')
im_truth = Image.open(tif_path)

truth_array = np.array(im_truth)

# Latest Prediction :
pred_path = join('/content/gdrive/My Drive/U-NET/EVALUATIONS','0.5Image.npy.png')
im_pred = Image.open(pred_path)

pred_array = np.array(im_pred)

In [ ]:
truth_array_0 = truth_array[:,:,0]
truth_array_1 = truth_array[:,:,1]
truth_array_2 = truth_array[:,:,2]
modif_array_0 = modif_array[:,:,0]

In [ ]:
# Precision formula :
(np.sum(pred_array==modif_array_0)/truth_array_0.size)*100

93.99477890114379

In [ ]:
# See the measurment_functions file
!cat measurment_functions.py

In [ ]:
# Returns nb_True_Negative, nb_True_Positive, nb_False_Positive, nb_False_Negative, accuracy, precision, recall, f_beta_score
import measurment_functions
beta = 1 # Change beta in function of the wanted results
measurment_functions.measure(beta, pred_array, truth_array_0)

Accuracy : 91.90421177151416%
Precision : 0.8843718985170695
Recall : 0.5887775525390675
F_beta Score : 0.7069185296439642 for beta = 1
nb_True_Negative = 772148
nb_True_Positive = 91781
nb_False_Positive = 12000
nb_False_Negative = 64103


(772148,
 91781,
 12000,
 64103,
 91.90421177151416,
 0.8843718985170695,
 0.5887775525390675,
 0.7069185296439642)